# IBM AI OpenScale & Custom machine learning engine

<img src="https://github.com/pmservice/ai-openscale-tutorials/raw/master/notebooks/images/image.png" align="left" alt="banner" width="100">


This notebook shows how to log the payload for the model deployed on custom model serving engine using AI OpenScale python client.

Contents
- [1. Setup](#setup)
- [2. Binding machine learning engine](#binding)
- [3. Subscriptions](#subscription)
- [4. Scoring and payload logging](#scoring)
- [5. Feedback logging](#feedback)
- [6. Data Mart](#datamart)

<a id="setup"></a>
## 1. Setup

### 1.0 Sample custom machine learning engine

The sample machine leraning engine based on docker image and deployment instructions can be found [here](https://github.com/pmservice/ai-openscale-tutorials/tree/master/applications/custom-ml-engine).

**NOTE:** CUSTOM machine learning engine must follow this [API specification](https://aiopenscale-custom-deployement-spec.mybluemix.net/) to be supported.

### 1.1 Installation and authentication

In [1]:
!pip install --upgrade ibm-ai-openscale

Requirement already up-to-date: ibm-ai-openscale in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages
Requirement not upgraded as not directly required: tabulate in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from ibm-ai-openscale)
Requirement not upgraded as not directly required: pandas in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from ibm-ai-openscale)
Requirement not upgraded as not directly required: h5py in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from ibm-ai-openscale)
Requirement not upgraded as not directly required: requests in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from ibm-ai-openscale)
Requirement not upgraded as not directly required: watson-machine-learning-client in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from ibm-ai-openscale)
Requirement not upgraded as not directly required: psycopg2 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from ibm-ai-openscale)
Requirement

Import and initiate.

In [51]:
from ibm_ai_openscale import APIClient
from ibm_ai_openscale.engines import *

### ACTION: Get `instance_guid` and `apikey`

How to get api key using bluemix console:
```
bx login --sso
bx iam api-key-create 'my_key'
```

How to get instance_guid (this is AI OpenScale instance GUID)
```
bx target -g <myResourceGroup>
bx resource service-instance ‘instance_name’
```


In [57]:
aios_credentials = {
  "url": "https://api.aiopenscale.cloud.ibm.com",
  "instance_guid": "***",
  "apikey": "***"
}

In [58]:
# The code was removed by Watson Studio for sharing.

In [59]:
client = APIClient(aios_credentials)

In [60]:
client.version

'1.0.190'

In [61]:
postgres_credentials = {
    "db_type": "postgresql",
    "uri_cli_1": "xxx",
    "maps": [],
    "instance_administration_api": {
        "instance_id": "xxx",
        "root": "xxx",
        "deployment_id": "xxx"
    },
    "name": "xxx",
    "uri_cli": "xxx",
    "uri_direct_1": "xxx",
    "ca_certificate_base64": "xxx",
    "deployment_id": "xxx",
    "uri": "xxx"
}

In [62]:
# The code was removed by Watson Studio for sharing.

Create schema for data mart.

In [63]:
schemaName = 'custom_engine_data_mart'

In [64]:
import psycopg2


hostname = postgres_credentials['uri'].split('@')[1].split(':')[0]
port = postgres_credentials['uri'].split('@')[1].split(':')[1].split('/')[0]
user = postgres_credentials['uri'].split('@')[0].split('//')[1].split(':')[0]
password = postgres_credentials['uri'].split('@')[0].split('//')[1].split(':')[1]
dbname = 'compose'

conn_string = "host=" + hostname + " port=" + port + " dbname=" + dbname + " user=" + user + " password=" + password
conn = psycopg2.connect(conn_string)
conn.autocommit = True
cursor = conn.cursor()
try:
    query = "drop schema " + schemaName + " cascade"
    cursor.execute(query)
except:
    pass
finally:    
    try:
        query = "create schema " + schemaName
        cursor.execute(query)
    finally:    
        conn.close()

### 1.2 DataMart setup

In [65]:
client.data_mart.setup(db_credentials=postgres_credentials, schema=schemaName)

In [66]:
data_mart_details = client.data_mart.get_details()

<a id="binding"></a>
## 2. Bind machine learning engines

### 2.1 Bind  `CUSTOM` machine learning engine
**NOTE:** CUSTOM machine learning engine must follow this [API specification](https://aiopenscale-custom-deployement-spec.mybluemix.net/) to be supported.

Credentials support following fields:
- `url` - list deployments endpoint url (required)
- `username` - part of BasicAuth (optional)
- `password` - part of BasicAuth (optional)

In [67]:
custom_engine_credentials = {
    "url": "***",
    "username": "***",
    "password": "***"
}

In [68]:
# The code was removed by Watson Studio for sharing.

In [69]:
binding_uid = client.data_mart.bindings.add('My custom engine', CustomMachineLearningInstance(custom_engine_credentials))

In [70]:
bindings_details = client.data_mart.bindings.get_details()

In [71]:
client.data_mart.bindings.list()

537cf17b-b171-48cf-9ddc-9e15dbfbf6d5,My custom engine,custom_machine_learning,2018-11-30T12:39:28.840Z


<a id="subsciption"></a>
## 3. Subscriptions

### 3.1 Add subscriptions

List available deployments.

In [72]:
client.data_mart.bindings.list_assets()

resnet50,resnet50,2016-12-01T10:11:12Z,model,,537cf17b-b171-48cf-9ddc-9e15dbfbf6d5,F
action,area and action prediction,2016-12-01T10:11:12Z,model,,537cf17b-b171-48cf-9ddc-9e15dbfbf6d5,F


In [73]:
subscription = client.data_mart.subscriptions.add(
    CustomMachineLearningAsset(source_uid='action', binding_uid=binding_uid, prediction_column='predictedActionLabel'))

#### Get subscriptions list

In [74]:
subscriptions = client.data_mart.subscriptions.get_details()

In [75]:
subscriptions_uids = client.data_mart.subscriptions.get_uids()
print(subscriptions_uids)

['action']


#### List subscriptions

In [76]:
client.data_mart.subscriptions.list()

action,area and action prediction,model,537cf17b-b171-48cf-9ddc-9e15dbfbf6d5,2018-11-30T12:49:27.456Z


### 3.2 Configure subscription

#### Enable payload logging in subscription

In [77]:
subscription.payload_logging.enable()

#### Get details of enabled payload logging

In [78]:
subscription.payload_logging.get_details()

{'enabled': True,
 'parameters': {'dynamic_schema_update': True,
  'table_name': 'custom_engine_data_mart.Payload_action'}}

<a id="scoring"></a>
# 4. Scoring and payload logging

## 4.1 Score the action model

In [79]:
import requests
import time


request_data = {'fields': ['ID',
                              'Gender',
                              'Status',
                              'Children',
                              'Age',
                              'Customer_Status',
                              'Car_Owner',
                              'Customer_Service',
                              'Business_Area',
                              'Satisfaction'],
                             'values': [[3785,
                               'Male',
                               'S',
                               1,
                               17,
                               'Inactive',
                               'Yes',
                               'The car should have been brought to us instead of us trying to find it in the lot.',
                               'Product: Information',
                               0]]}

header = {'Content-Type': 'application/json'}
scoring_url = subscription.get_details()['entity']['deployments'][0]['scoring_endpoint']['url']

start_time = time.time()
response = requests.post(scoring_url, json=request, headers=header)
response_time = int((time.time() - start_time)*1000)

response_data = r.json()
print('Response: ' + str(response_data))

Response: {'values': [[3785, 'Male', 'S', 1, 17, 'Inactive', 'Yes', 'The car should have been brought to us instead of us trying to find it in the lot.', 'Product: Information', 0, ['the', 'car', 'should', 'have', 'been', 'brought', 'to', 'us', 'instead', 'of', 'us', 'trying', 'to', 'find', 'it', 'in', 'the', 'lot.'], [262144.0, [9639.0, 21872.0, 74079.0, 86175.0, 91878.0, 99585.0, 103838.0, 175817.0, 205044.0, 218965.0, 222453.0, 227152.0, 227431.0, 229772.0, 253475.0], [1.0, 2.0, 1.0, 1.0, 1.0, 1.0, 2.0, 1.0, 2.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]], [262144.0, [9639.0, 21872.0, 74079.0, 86175.0, 91878.0, 99585.0, 103838.0, 175817.0, 205044.0, 218965.0, 222453.0, 227152.0, 227431.0, 229772.0, 253475.0], [1.7479568465696622, 5.81081927052141, 0.0, 1.5191152741408147, 3.5115454388310208, 3.5115454388310208, 1.2986891158031049, 0.0, 1.563032662020071, 0.0, 1.8375690052593492, 0.0, 3.5115454388310208, 1.1444218246994038, 1.9021075263969205]], 7.0, [45.0, 48.0, 5.0, 10.0, 7.0, 6.0, 8.0, 3.0], 

## 4.2 Store the request and response in payload logging table

### Python SDK

In [80]:
from ibm_ai_openscale.supporting_classes import PayloadRecord

records_list = [PayloadRecord(request=request_data, response=response_data, response_time=response_time), 
                PayloadRecord(request=request_data, response=response_data, response_time=response_time)]

subscription.payload_logging.store(records=records_list)

### REST API

Get the token first.

In [81]:
token_endpoint = "https://iam.bluemix.net/identity/token"
headers = {
    "Content-Type": "application/x-www-form-urlencoded",
    "Accept": "application/json"
}

data = {
    "grant_type":"urn:ibm:params:oauth:grant-type:apikey",
    "apikey":aios_credentials["apikey"]
}

req = requests.post(token_endpoint, data=data, headers=headers)
token = req.json()['access_token']

In [82]:
import requests, uuid

PAYLOAD_STORING_HREF_PATTERN = '{}/v1/data_marts/{}/scoring_payloads'
endpoint = PAYLOAD_STORING_HREF_PATTERN.format(aios_credentials['url'], aios_credentials['data_mart_id'])

payload = [{
    'binding_id': binding_uid, 
    'deployment_id': subscription.get_details()['entity']['deployments'][0]['deployment_id'], 
    'subscription_id': subscription.uid, 
    'scoring_id': str(uuid.uuid4()), 
    'response': response_data,
    'request': request_data
}]


headers = {"Authorization": "Bearer " + token}
      
req_response = requests.post(endpoint, json=payload, headers = headers)

print("Request OK: " + str(req_response.ok))

Request OK: True


<a id="datamart"></a>
# 5. Feedback logging & quality (accuracy) monitoring

Enable quality monitoring.

In [86]:
subscription.quality_monitoring.enable(threshold=0.7, min_records=10)

In [88]:
subscription.feedback_logging.print_table_schema()

ID,integer,True
Gender,string,True
Status,string,True
Children,integer,True
Age,integer,True
Customer_Status,string,True
Car_Owner,string,True
Customer_Service,string,True
Business_Area,string,True
Satisfaction,integer,True
label,string,True


In [92]:
fields = ['ID', 'Gender', 'Status','Children', 'Age', 'Customer_Status', 'Car_Owner', 'Customer_Service', 'Business_Area', 'Satisfaction', 'label']

records = [
    [3785, 'Male', 'S', 1, 17,'Inactive', 'Yes', 'The car should have been brought to us instead of us trying to find it in the lot.', 'Product: Information', 0, 'On-demand pickup location'],
    [3785, 'Male', 'S', 1, 17,'Inactive', 'Yes', 'The car should have been brought to us instead of us trying to find it in the lot.', 'Product: Information', 0, 'On-demand pickup location']]



subscription.feedback_logging.store(feedback_data=records, fields=fields)

<a id="datamart"></a>
# 6. Get the logged data

#### Print schema of payload_logging table

In [48]:
subscription.payload_logging.print_table_schema()

scoring_id,string,False
scoring_timestamp,timestamp,False
deployment_id,string,False
asset_revision,string,True
ID,integer,True
Gender,string,True
Status,string,True
Children,integer,True
Age,integer,True
Customer_Status,string,True
Car_Owner,string,True


#### Show (preview) the table

In [49]:
subscription.payload_logging.show_table()

d8c602df-fd79-44fd-80fc-5f383c569954-1,2018-11-30 12:10:37.154000+00:00,action,None,3785,Male,S,1,17,Inactive,Yes,The car should have been brought to us instead of us trying to find it in the lot.,Product: Information,0,"['the', 'car', 'should', 'have', 'been', 'brought', 'to', 'us', 'instead', 'of', 'us', 'trying', 'to', 'find', 'it', 'in', 'the', 'lot.']","[262144.0, [9639.0, 21872.0, 74079.0, 86175.0, 91878.0, 99585.0, 103838.0, 175817.0, 205044.0, 218965.0, 222453.0, 227152.0, 227431.0, 229772.0, 253475.0], [1.0, 2.0, 1.0, 1.0, 1.0, 1.0, 2.0, 1.0, 2.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]]","[262144.0, [9639.0, 21872.0, 74079.0, 86175.0, 91878.0, 99585.0, 103838.0, 175817.0, 205044.0, 218965.0, 222453.0, 227152.0, 227431.0, 229772.0, 253475.0], [1.7479568465696622, 5.81081927052141, 0.0, 1.5191152741408147, 3.5115454388310208, 3.5115454388310208, 1.2986891158031049, 0.0, 1.563032662020071, 0.0, 1.8375690052593492, 0.0, 3.5115454388310208, 1.1444218246994038, 1.9021075263969205]]",7,"[45.0, 48.0, 5.0, 10.0, 7.0, 6.0, 8.0, 3.0]","[0.3409090909090909, 0.36363636363636365, 0.03787878787878788, 0.07575757575757576, 0.05303030303030303, 0.045454545454545456, 0.06060606060606061, 0.022727272727272728]",1,Product: Functioning,0,1,1,1,"[0.0, 1.0, 1.0, 1.0, 1.0, 17.0, 0.0, 1.0]","[0.0, 1.0, 6.0, 0.0, 0.0]","[0.0, 0.14285714285714285, 0.8571428571428571, 0.0, 0.0]",2,On-demand pickup location
35c71a17-7222-4dbc-a8d5-f4f951ad6049-1,2018-11-30 11:29:28.676989+00:00,action,None,3785,Male,S,1,17,Inactive,Yes,The car should have been brought to us instead of us trying to find it in the lot.,Product: Information,0,"['the', 'car', 'should', 'have', 'been', 'brought', 'to', 'us', 'instead', 'of', 'us', 'trying', 'to', 'find', 'it', 'in', 'the', 'lot.']","[262144.0, [9639.0, 21872.0, 74079.0, 86175.0, 91878.0, 99585.0, 103838.0, 175817.0, 205044.0, 218965.0, 222453.0, 227152.0, 227431.0, 229772.0, 253475.0], [1.0, 2.0, 1.0, 1.0, 1.0, 1.0, 2.0, 1.0, 2.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]]","[262144.0, [9639.0, 21872.0, 74079.0, 86175.0, 91878.0, 99585.0, 103838.0, 175817.0, 205044.0, 218965.0, 222453.0, 227152.0, 227431.0, 229772.0, 253475.0], [1.7479568465696622, 5.81081927052141, 0.0, 1.5191152741408147, 3.5115454388310208, 3.5115454388310208, 1.2986891158031049, 0.0, 1.563032662020071, 0.0, 1.8375690052593492, 0.0, 3.5115454388310208, 1.1444218246994038, 1.9021075263969205]]",7,"[45.0, 48.0, 5.0, 10.0, 7.0, 6.0, 8.0, 3.0]","[0.3409090909090909, 0.36363636363636365, 0.03787878787878788, 0.07575757575757576, 0.05303030303030303, 0.045454545454545456, 0.06060606060606061, 0.022727272727272728]",1,Product: Functioning,0,1,1,1,"[0.0, 1.0, 1.0, 1.0, 1.0, 17.0, 0.0, 1.0]","[0.0, 1.0, 6.0, 0.0, 0.0]","[0.0, 0.14285714285714285, 0.8571428571428571, 0.0, 0.0]",2,On-demand pickup location
bb7f7fdd-b323-4e4f-b2e8-6714279b7f83-1,2018-11-30 11:29:28.676915+00:00,action,None,3785,Male,S,1,17,Inactive,Yes,The car should have been brought to us instead of us trying to find it in the lot.,Product: Information,0,"['the', 'car', 'should', 'have', 'been', 'brought', 'to', 'us', 'instead', 'of', 'us', 'trying', 'to', 'find', 'it', 'in', 'the', 'lot.']","[262144.0, [9639.0, 21872.0, 74079.0, 86175.0, 91878.0, 99585.0, 103838.0, 175817.0, 205044.0, 218965.0, 222453.0, 227152.0, 227431.0, 229772.0, 253475.0], [1.0, 2.0, 1.0, 1.0, 1.0, 1.0, 2.0, 1.0, 2.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]]","[262144.0, [9639.0, 21872.0, 74079.0, 86175.0, 91878.0, 99585.0, 103838.0, 175817.0, 205044.0, 218965.0, 222453.0, 227152.0, 227431.0, 229772.0, 253475.0], [1.7479568465696622, 5.81081927052141, 0.0, 1.5191152741408147, 3.5115454388310208, 3.5115454388310208, 1.2986891158031049, 0.0, 1.563032662020071, 0.0, 1.8375690052593492, 0.0, 3.5115454388310208, 1.1444218246994038, 1.9021075263969205]]",7,"[45.0, 48.0, 5.0, 10.0, 7.0, 6.0, 8.0, 3.0]","[0.3409090909090909, 0.36363636363636365, 0.03787878787878788, 0.07575757575757576, 0.05303030303030303, 0.045454545454545456, 0.06060606060606061, 0.02272

#### Return the table content as pandas dataframe

In [50]:
pandas_df = subscription.payload_logging.get_table_content(format='pandas')
pandas_df

,scoring_id,scoring_timestamp,deployment_id,asset_revision,ID,Gender,Status,Children,Age,Customer_Status,...,predictedAreaLabel,gender_ix,customer_status_ix,status_ix,owner_ix,features,rawPrediction,probability,prediction,predictedActionLabel
0,d8c602df-fd79-44fd-80fc-5f383c569954-1,2018-11-30 12:10:37.154000+00:00,action,None,3785,Male,S,1,17,Inactive,...,Product: Functioning,0,1,1,1,"[0.0, 1.0, 1.0, 1.0, 1.0, 17.0, 0.0, 1.0]","[0.0, 1.0, 6.0, 0.0, 0.0]","[0.0, 0.14285714285714285, 0.8571428571428571,...",2,On-demand pickup location
1,35c71a17-7222-4dbc-a8d5-f4f951ad6049-1,2018-11-30 11:29:28.676989+00:00,action,None,3785,Male,S,1,17,Inactive,...,Product: Functioning,0,1,1,1,"[0.0, 1.0, 1.0, 1.0, 1.0, 17.0, 0.0, 1.0]","[0.0, 1.0, 6.0, 0.0, 0.0]","[0.0, 0.14285714285714285, 0.8571428571428571,...",2,On-demand pickup location
2,bb7f7fdd-b323-4e4f-b2e8-6714279b7f83-1,2018-11-30 11:29:28.676915+00:00,action,None,3785,Male,S,1,17,Inactive,...,Product: Functioning,0,1,1,1,"[0.0, 1.0, 1.0, 1.0, 1.0, 17.0, 0.0, 1.0]","[0.0, 1.0, 6.0, 0.0, 0.0]","[0.0, 0.14285714285714285, 0.8571428571428571,...",2,On-demand pickup location


In [93]:
subscription.feedback_logging.show_table()

3785,Male,S,1,17,Inactive,Yes,The car should have been brought to us instead of us trying to find it in the lot.,Product: Information,0,On-demand pickup location,2018-11-30 12:55:10.598000+00:00
3785,Male,S,1,17,Inactive,Yes,The car should have been brought to us instead of us trying to find it in the lot.,Product: Information,0,On-demand pickup location,2018-11-30 12:55:10.598000+00:00
3785,Male,S,1,17,Inactive,Yes,The car should have been brought to us instead of us trying to find it in the lot.,Product: Information,0,On-demand pickup location,2018-11-30 12:55:39.738000+00:00
3785,Male,S,1,17,Inactive,Yes,The car should have been brought to us instead of us trying to find it in the lot.,Product: Information,0,On-demand pickup location,2018-11-30 12:55:39.738000+00:00


---

### Authors
Lukasz Cmielowski, PhD, is an Automation Architect and Data Scientist at IBM with a track record of developing enterprise-level applications that substantially increases clients' ability to turn data into actionable knowledge.